# 2D pattern matching

Zadanie dotyczy wyszukiwania wzorców dwuwymiarowych.

1. Zaimplementuj algorytm wyszukiwania wzorca 2-wymiarowego 
2. Znajdź w załączonym pliku "haystack.txt" wszyskie sytuacje, gdy taka sama litera występuje na tej samej pozycji w dwóch kolejnych linijkach. Zwróć uwagę, na nierówną długość linii w pliku. 
3. Znajdź wszystkie wystąpienia "th" oraz "t h" w dwóch kolejnych liniach na tej samej pozycji. 
4. Wybierz przynajmniej 4 litery (małe). Znajdź wszystkie wystąpienia tej litery w załączonym pliku "haystack.png" 
5. Znajdź wszystkie wystąpienia słowa "p a t t e r n" w haystack.png. 
6. Porównaj czas budowania automatu i czas wyszukiwania dla różnych rozmiarów wzorca 
7. Podziel plik na 2, 4 i 8 fragmentów (w poziomie) i porównaj czas przeszukiwania 

Załączone pliki to fragmenty książki "Jewels of Stringology".

In [1]:
from collections import deque
from collections import defaultdict
import random
from PIL import Image
from time import perf_counter

In [2]:
class Node:
    def __init__(self, letter=None, parent=None, terminal=False):
        self.letter = letter
        self.parent = parent
        self.terminal = terminal
        self.children = {}
        self.suffix_link = None
        self.dict_link = None
        self.word = self._build_word()

    def _build_word(self):
        if self.parent is None:
            return ''
        else:
            return self.parent.word + self.letter

    def __repr__(self):
        return self.word

### Ex 1

In [3]:
class AhoCorasick:
    def __init__(self, patterns):
        self.root = Node()
        self.pattern_idx = self.build_pattern_idx(patterns)
        self.build_trie(patterns)
        self.build_suffix_links()
        self.build_dict_links()
        self.search_time = None

    def build_pattern_idx(self, patterns):
        pattern_idx = defaultdict(list)
        for idx, pattern in enumerate(patterns):
            pattern_idx[pattern] = idx
        return pattern_idx

    def build_trie(self, patterns):
        for pattern in patterns:
            node = self.root
            for idx, letter in enumerate(pattern):
                terminal = (idx == len(pattern) - 1)
                if letter in node.children:
                    node = node.children[letter]
                    if terminal:
                        node.terminal = True
                else:
                    new_node = Node(letter, parent=node, terminal=terminal)
                    node.children[letter] = new_node
                    node = new_node

    def build_suffix_links(self):
        queue = deque()
        queue.append(self.root)
        while queue:
            node = queue.popleft()
            for letter, child in node.children.items():
                queue.append(child)
                if node == self.root:
                    child.suffix_link = self.root
                else:
                    suffix = node.suffix_link
                    while suffix is not None and letter not in suffix.children:
                        suffix = suffix.suffix_link
                    if suffix is None:
                        child.suffix_link = self.root
                    else:
                        child.suffix_link = suffix.children[letter]

    def build_dict_links(self):
        queue = deque()
        queue.append(self.root)
        while queue:
            node = queue.popleft()
            for letter, child in node.children.items():
                queue.append(child)
                suffix = child.suffix_link
                while suffix is not None and not suffix.terminal:
                    suffix = suffix.suffix_link
                if suffix is None:
                    child.dict_link = self.root
                else:
                    child.dict_link = suffix

    def search(self, text):
        node = self.root
        result = []
        result_idx = ['#' for _ in range(len(text))]
        for idx, letter in enumerate(text):
            while node is not None and letter not in node.children:
                node = node.suffix_link
            if node is None:
                node = self.root
            else:
                node = node.children[letter]
            if node is None:
                node = self.root
            else:
                temp = node
                while temp is not None:
                    if temp.terminal:
                        result.append((idx - len(temp.word) + 1, temp.word))
                        result_idx[idx - len(temp.word) + 1] = self.pattern_idx[temp.word]
                    temp = temp.dict_link
        # result_idx = ''.join([str(idx) for idx in result_idx])
        return (result, result_idx)

In [4]:
def align_text(text, splited=True):
    if not splited:
        lines = text.split('\n')
    else:
        lines = text
    max_length = max(len(line) for line in lines)
    aligned_lines = [line.ljust(max_length) for line in lines]
    return aligned_lines

def split_columns(matrix):
    result = []
    splited = [list(column) for column in zip(*matrix)]
    for column in splited:
        result.append(''.join(column))
    return result
        

In [5]:
def pattern_matching_2d(text, pattern, splited=True, time=False):
    if time:
        start = perf_counter()
    # przygotowanie patternu
    pattern = split_columns(align_text(pattern))
    ac_vertical = AhoCorasick(pattern)
    if time:
        end = perf_counter()
        ac_ver_time = end - start
        start = perf_counter()
    # pattern do znalezienia poziomo
    horizontal_pattern = ""
    for column in pattern:
        horizontal_pattern += str(ac_vertical.pattern_idx[column])
    
    # # przygotowanie tekstu
    text = split_columns(align_text(text, splited=splited))
    result = []
    for line in text:
        _, res = ac_vertical.search(line)
        result.append(res)
    
    # w tym szukamy poziomo
    transposed = [[] for _ in range(len(result[0]))]
    for line in result:
        for idx, element in enumerate(line):
            transposed[idx].append(element)

    temp = []
    for line in transposed:
        temp.append(''.join([str(element) for element in line]))
    transposed = '\n'.join(temp)

    ac_horizontal = AhoCorasick([horizontal_pattern])
    result = []
    for idx, line in enumerate(transposed.split('\n')):
        res, _ = ac_horizontal.search(line)
        for found in res:
            result.append((idx, found[0]))
    if time:
        end = perf_counter()
        return ac_ver_time, end - start, result
    return result


### Ex 2

In [6]:
with open('haystack.txt', 'r') as f:
    text = f.read()

In [7]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

for letter in alphabet:
    pattern = [letter, letter]
    result = pattern_matching_2d(text, pattern)
    print("PATTERN: ", pattern)
    print("NUMBER: ", len(result))
    print(result)
    print("--------------------------------")


PATTERN:  ['a', 'a']
NUMBER:  0
[]
--------------------------------
PATTERN:  ['b', 'b']
NUMBER:  0
[]
--------------------------------
PATTERN:  ['c', 'c']
NUMBER:  9
[(529, 0), (1051, 0), (2169, 0), (3353, 0), (3374, 0), (3486, 0), (5155, 0), (6179, 0), (6252, 0)]
--------------------------------
PATTERN:  ['d', 'd']
NUMBER:  0
[]
--------------------------------
PATTERN:  ['e', 'e']
NUMBER:  5
[(2156, 0), (4750, 0), (5769, 0), (5981, 0), (6091, 0)]
--------------------------------
PATTERN:  ['f', 'f']
NUMBER:  8
[(2035, 0), (2632, 0), (3696, 0), (3799, 0), (3999, 0), (4729, 0), (5844, 0), (6063, 0)]
--------------------------------
PATTERN:  ['g', 'g']
NUMBER:  0
[]
--------------------------------
PATTERN:  ['h', 'h']
NUMBER:  0
[]
--------------------------------
PATTERN:  ['i', 'i']
NUMBER:  0
[]
--------------------------------
PATTERN:  ['j', 'j']
NUMBER:  0
[]
--------------------------------
PATTERN:  ['k', 'k']
NUMBER:  0
[]
--------------------------------
PATTERN:  ['l', '

### Ex 3

In [8]:
patterns = [["th", "th"], ["t h", "t h"]]

for pattern in patterns:
    result = pattern_matching_2d(text, pattern)
    print("PATTERN: ", pattern)
    print("NUMBER: ", len(result))
    print(result)
    print("--------------------------------")

PATTERN:  ['th', 'th']
NUMBER:  0
[]
--------------------------------
PATTERN:  ['t h', 't h']
NUMBER:  0
[]
--------------------------------


### Ex 4

In [9]:
def image_to_matrix(file_name):
    '''
    nie jestem dumny z tej funkcji :c
    '''
    image = Image.open(file_name)
    pixels = list(image.getdata())
    width, height = image.size
    text = []
    i = width
    for pixel in pixels:
        if i == width:
            i = 0
            text.append([])

        # wyrównanie do 3 znaków
        if pixel[0] < 10:
            pixel_val = '00' + str(pixel[0])
        elif pixel[0] < 100:
            pixel_val = '0' + str(pixel[0])
        else:
            pixel_val = str(pixel[0])

        text[-1].append(pixel_val)
        i += 1
    result = []
    for line in text:
        line = ''.join(line)
        result.append(line)
    return result

In [10]:
a_img = image_to_matrix('img/a.png')
m_img = image_to_matrix('img/m.png')
p_img = image_to_matrix('img/p.png')
s_img = image_to_matrix('img/s.png')
haystack_img = image_to_matrix('haystack.png')

haystack_img = '\n'.join(haystack_img)

In [11]:
print("PATTERN: a")
result = pattern_matching_2d(haystack_img, a_img, splited=False)
print("NUMBER: ", len(result))
print(result)
print("--------------------------------")

PATTERN: a
NUMBER:  356
[(36, 834), (36, 1085), (36, 1373), (36, 2012), (36, 2516), (36, 3029), (58, 1226), (58, 1379), (58, 1636), (58, 2677), (58, 2946), (80, 716), (80, 839), (80, 1612), (80, 2279), (80, 2396), (102, 272), (102, 490), (102, 1204), (124, 1125), (124, 1345), (124, 2445), (146, 540), (146, 693), (146, 1071), (146, 1500), (146, 2243), (146, 2567), (168, 284), (168, 455), (168, 892), (168, 1424), (168, 2187), (168, 2357), (168, 2477), (168, 2965), (190, 351), (190, 1022), (190, 1596), (190, 2274), (190, 2453), (190, 2694), (212, 1712), (212, 2104), (234, 501), (234, 1895), (234, 2373), (256, 426), (256, 1756), (278, 289), (278, 580), (278, 1178), (278, 1964), (278, 2373), (300, 189), (322, 434), (322, 776), (322, 1787), (322, 2041), (322, 2345), (344, 430), (344, 893), (344, 1110), (344, 1668), (344, 2336), (366, 150), (366, 1007), (366, 1130), (366, 2415), (366, 2626), (388, 507), (388, 991), (388, 1545), (388, 2875), (410, 150), (432, 979), (432, 1135), (432, 2021), (4

In [12]:
print("PATTERN: m")
result = pattern_matching_2d(haystack_img, m_img, splited=False)
print("NUMBER: ", len(result))  
print(result)
print("--------------------------------")

PATTERN: m
NUMBER:  131
[(37, 439), (37, 1501), (37, 2328), (103, 1887), (125, 790), (125, 1752), (147, 1116), (147, 2108), (169, 339), (169, 1259), (169, 1635), (191, 212), (191, 1608), (191, 1697), (213, 1551), (235, 150), (235, 1024), (235, 1729), (257, 420), (257, 992), (257, 1581), (279, 771), (323, 775), (323, 1746), (345, 627), (345, 1498), (345, 1620), (345, 1822), (345, 2155), (367, 696), (367, 1803), (389, 220), (389, 678), (389, 1123), (411, 213), (433, 625), (455, 1204), (477, 289), (477, 934), (477, 1427), (477, 1806), (499, 302), (499, 2113), (521, 1168), (543, 507), (543, 1738), (543, 1998), (565, 1340), (587, 2295), (653, 878), (653, 1200), (653, 1510), (653, 2197), (675, 2029), (697, 324), (719, 624), (741, 833), (741, 2075), (785, 1197), (785, 1286), (785, 1919), (807, 1131), (807, 1762), (807, 2151), (829, 1228), (851, 659), (851, 1884), (851, 2255), (873, 1943), (895, 249), (895, 495), (917, 174), (939, 141), (939, 230), (1005, 78), (1005, 2030), (1027, 78), (1027, 

In [13]:
print("PATTERN: p")
result = pattern_matching_2d(haystack_img, p_img, splited=False)
print("NUMBER: ", len(result))
print(result)
print("--------------------------------")

PATTERN: p
NUMBER:  131
[(37, 480), (37, 1145), (37, 1738), (59, 723), (103, 1399), (125, 78), (125, 943), (125, 2054), (147, 1193), (147, 1621), (191, 414), (191, 1390), (213, 419), (235, 1047), (257, 791), (257, 1613), (279, 1193), (323, 626), (323, 1545), (345, 2085), (367, 510), (367, 1403), (367, 1936), (367, 1989), (389, 1102), (411, 419), (433, 444), (477, 333), (477, 575), (477, 1200), (499, 1299), (521, 656), (543, 732), (543, 2015), (587, 437), (587, 1076), (631, 761), (653, 381), (653, 650), (653, 2134), (675, 804), (675, 1028), (675, 2102), (719, 1055), (719, 1393), (763, 105), (763, 450), (785, 342), (785, 1023), (807, 132), (807, 964), (807, 1731), (807, 2194), (829, 1030), (829, 1083), (851, 1913), (873, 915), (895, 78), (917, 228), (917, 1357), (939, 422), (939, 628), (939, 1296), (961, 551), (961, 604), (961, 1295), (961, 2036), (983, 216), (983, 569), (983, 1566), (983, 1987), (983, 2040), (1027, 1818), (1049, 78), (1071, 650), (1071, 1452), (1093, 99), (1093, 1135), 

In [14]:
print("PATTERN: s")
result = pattern_matching_2d(haystack_img, s_img, splited=False)
print("NUMBER: ", len(result))
print(result)
print("--------------------------------")

PATTERN: s
NUMBER:  334
[(37, 378), (37, 569), (37, 1201), (37, 1797), (37, 2093), (37, 2389), (59, 462), (59, 620), (59, 1093), (81, 564), (81, 806), (81, 1417), (81, 1860), (103, 93), (103, 785), (103, 1543), (103, 1584), (103, 1748), (103, 1822), (103, 1971), (125, 492), (125, 1328), (125, 1552), (125, 1626), (125, 1775), (125, 1867), (125, 2205), (125, 2246), (147, 243), (147, 1349), (147, 2005), (169, 774), (169, 1238), (169, 1819), (169, 1947), (191, 555), (191, 596), (191, 1201), (191, 2004), (213, 273), (213, 572), (213, 613), (213, 861), (213, 1619), (213, 1747), (213, 2010), (235, 432), (235, 701), (235, 1702), (235, 1776), (235, 1883), (257, 486), (257, 566), (257, 1015), (257, 1722), (279, 162), (279, 551), (279, 2038), (323, 297), (323, 914), (323, 1258), (323, 2064), (345, 111), (345, 434), (345, 475), (345, 1410), (345, 2198), (367, 183), (367, 473), (367, 805), (367, 1239), (367, 1703), (389, 78), (389, 152), (389, 943), (389, 1473), (389, 1532), (389, 1639), (389, 1803

### Ex 5

In [15]:
pattern_img = image_to_matrix('img/pattern.png')

print("PATTERN: p a t t e r n")
result = pattern_matching_2d(haystack_img, pattern_img, splited=False)
print("NUMBER: ", len(result))
print(result)
print("--------------------------------")

PATTERN: p a t t e r n
NUMBER:  5
[(475, 1350), (497, 2850), (541, 1897), (585, 1104), (629, 1673)]
--------------------------------


### Ex 6

In [16]:
def time_test(text, pattern):
    ac_time, se_time, _ = pattern_matching_2d(text, pattern, time=True)
    print("Build automaton", ac_time)
    print("Pattern matching", se_time)

In [17]:
with open('haystack.txt', 'r') as f:
    text = f.read()

text = align_text(text, splited=False)
n = min(len(text), len(text[0]))

In [18]:
sizes = [1, 5, 10, 20, 50, 80]
patterns = []

for size in sizes:
    pattern = []
    i, j = random.randint(0, n - size- 1), random.randint(0, n - size - 1)
    pattern = text[i:i+size][j:j+size]
    for i in range(i, i + size):
        pattern.append(text[i][j:j+size])
    patterns.append(pattern)

In [19]:
for idx, pattern in enumerate(patterns):
    print("SIZE: ", sizes[idx])
    time_test(text, pattern)
    print("--------------------------------")

SIZE:  1
Build automaton 3.619998460635543e-05
Pattern matching 0.006125600019004196
--------------------------------
SIZE:  5
Build automaton 0.00010900001507252455
Pattern matching 0.006620300002396107
--------------------------------
SIZE:  10
Build automaton 0.000271500030066818
Pattern matching 0.006362599960993975
--------------------------------
SIZE:  20
Build automaton 0.000846999988425523
Pattern matching 0.008028200012631714
--------------------------------
SIZE:  50
Build automaton 0.025040100037585944
Pattern matching 0.006980100006330758
--------------------------------
SIZE:  80
Build automaton 0.04483930004062131
Pattern matching 0.006491499952971935
--------------------------------


### Ex 7

In [20]:
def div_time_test(text, pattern):
    result = []
    for div in [2, 4, 8]:
        length = len(text) // div
        intervals = [text[i * length:(i + 1) * length] for i in range(div)]
        start = perf_counter()
        for i in intervals:
            pattern_matching_2d(i, pattern)
        end = perf_counter()
        print("Div", div, "\nTime", end - start)

In [21]:
div_time_test(text, patterns[0])

Div 2 
Time 0.005861399986315519
Div 4 
Time 0.0077729999902658165
Div 8 
Time 0.006117100012488663
